<a href="https://colab.research.google.com/github/Khotso-Bore/Local-Recoding-Anonymization/blob/Innocentia's/local_recoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pprint import pprint

import pandas as pd
import numpy as np

### Prepare Dataset

In [ ]:
df = pd.read_csv('adult/adult.data', header=None,names=[
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education-num",
    "marital-status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital-gain",
    "capital-loss",
    "hours-per-week",
    "native-country",
    "income"
]
 )

In [ ]:
df.head()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

### drop coloumns

In [ ]:
drop_columns = ['capital-gain', 'capital-loss', 'fnlwgt', 'education-num','income']

'''
# Drop unnecessary columns
These columns are dropped as they are not needed as
they may not contain any sensitive information.
required for the local recoding anonymization process.
'''
df.drop(columns=drop_columns, inplace=True)

In [ ]:
# count number of columns
print(f"Number of columns after dropping unnecessary columns: {len(df.columns)}")

In [ ]:
df.info()

In [ ]:
print("Sensitive Attribute - workclass value counts:")
df['workclass'].value_counts()

In [ ]:
# Show the first row where 'workclass' contains '?'
missing_value = df[df['workclass'].str.contains('?', regex=False)]['workclass'].head(1).values[0]
print(f"Missing value representation in 'workclass': {missing_value}")

In [ ]:
df.replace(missing_value,np.nan,inplace=True)
df.dropna(inplace=True)
df['workclass'].value_counts()



In [ ]:
df['marital-status'].value_counts()

In [ ]:
#plot the distribution of every column
import matplotlib.pyplot as plt
for column in df.columns:
    plt.figure(figsize=(10,5))
    df[column].value_counts().plot(kind='bar')
    plt.title(f'Distribution of {column}')
    plt.xlabel(column)
    plt.ylabel('Count')
    plt.show()

# Algorithm 2

In [ ]:
#save unqiq values of each column to as a string array
unique_values = {}
for column in df.columns:
    unique_values[column] = df[column].unique().tolist()
    print(f"Unique values in {column}: {unique_values[column]}")


In [ ]:
import json

with open('taxonomy-tree.json', 'r') as f:
    taxonomy_dict = json.load(f)

# Display the dictionary
taxonomy_dict

In [ ]:
#invert the taxonomy dictionary to get child to parent mapping
def invert_taxonomy_tree(taxonomy, path=[]):
    inverted_taxonomy_tree = {}
    for key in taxonomy:


        extended_path = [key] + path
        # if(key == "workclass"):
        #     print(taxonomy[key])
        #     print(path)
        # print(f"Current key: {key}, Current path: {path}")
        if isinstance(taxonomy[key], dict):
            result = invert_taxonomy_tree(taxonomy[key], extended_path)
            inverted_taxonomy_tree.update(result)
            # if(key == "workclass"):
            #     print(inverted_taxonomy_tree)

        if isinstance(taxonomy[key], list):
            for item in taxonomy[key]:
                inverted_taxonomy_tree[item] = [item] + extended_path

        # path = []
    return inverted_taxonomy_tree

inverted_taxonomy_tree = invert_taxonomy_tree(taxonomy_dict, [])
for key in inverted_taxonomy_tree:
    print(f"{key}: {inverted_taxonomy_tree[key]}")


## Provenence Set

In [ ]:
def provenance(values):
    result = []
    for value in values:
        mapping = inverted_taxonomy_tree.get(value, None)
        if mapping:
            result = result + mapping[:-1]  # Exclude the original value
    return result


In [ ]:
df.head(1).values[0]

In [ ]:

first_row = list(df.head(1).values[0])
first_row = [s.strip() for s in first_row if isinstance(s, str)]


In [ ]:
provenance(first_row)

In [ ]:
education_tree = {
        "Secondary": {
            "Junior": ["9th", "10th"],
            "Senior": ["11th", "12th"]
        },
        "University": {
            "Bachelor": ["Bachelor"],
            "Graduate": ["Master", "Doctorate"]
        }
}

# education_tree = {
#     "Any_Education": ["hello"]
# }


## Algorith 2 characteristic_vector_converting

In [ ]:
def characteristic_vector_converting(taxonomy_tree, values, inverted_taxonomy_tree):
    characteristic_vector = []
    # print(f"Values to process for characteristic vector: {values}")
    for val in values:
        # print(f"Processing value: {val}")
        val = val.strip()
        last_value = inverted_taxonomy_tree[val][-1]
        # print(f"Processing value: {val}, Last value in taxonomy path: {last_value}")
        # Get the values in the inverted taxonomy tree that have key as the last value
        arr = []
        for k in inverted_taxonomy_tree:
            mapping = inverted_taxonomy_tree[k]
            if mapping and mapping[-1] == last_value:
                arr.append(mapping[:-1])  # Exclude the original value

        arr = np.array(arr)
        # print(arr.shape)
        # print(arr)
        # for each column in arr
        # print(arr.shape[1])
        provenance_set = provenance([val])
        for col in range(len(provenance_set)):  # every column except last column
            # Get the unique values in the column
            coloumn_values = arr[:, col]
            unique_values = np.unique(coloumn_values)
            # print(f"Unique values in column {provenance_set[col]}: {unique_values}")
            # print(f"Value to encode: {unique_values[0]}")
            vector = [0] * len(unique_values)
            # print(val)
            index = np.where(unique_values == provenance_set[col])[0][0]
            # print(f"Index of value {val} in unique values: {index}")
            vector[index] = 1
            characteristic_vector.extend(vector)
            # Do something with the vector
    return characteristic_vector


In [ ]:
first_row[-2:]

In [ ]:
characteristic_vector_converting(taxonomy_dict, first_row, inverted_taxonomy_tree)

# Experiment parameters

In [ ]:
import random

In [ ]:
# k=??
# theta=??
alpha=50

# Creating set of hash functions (universal use)

In [ ]:

# creates set F of hash functions Hash functions in F are in
#  the form of h(x)=(ax+b) mod NPrime, where a and b are
#  random integers, and NPrime is the smallest prime number
#  larger than |U|.

# F is a list of hash functions
# Each hash function h takes an input x (row index in the characteristic vector)
# and outputs a hashed value modulo a large prime number.
# Example: h(x) = (a * x + b) % N_prime
U=len(characteristic_vector_converting(taxonomy_dict, first_row, inverted_taxonomy_tree))
random.seed(42)
def create_hash_fam(num_hashes,U_size):
  def is_prime(n):
    if n<2:
      return False

    for i in range(2,int(n**0.5)+1):
      if n%i==0:
        return False
    return True

  def next_prime(n):
    while not is_prime(n):
      n=n+1
    return n

  N_prime=next_prime(U_size +1)

  F=[]

  for i in range(num_hashes):
    a=random.randint(1,N_prime-1)
    b=random.randint(0,N_prime-1)
    h = lambda x, a=a, b=b, N_prime=N_prime: (a * x + b) % N_prime
    F.append(h)
  return F
F = create_hash_fam(num_hashes=alpha, U_size=U)

# Algorithm 4

In [ ]:
import random

In [ ]:
from math import inf
###ALGORITHM 4
def minhash(characteristic_vec,h_ab):
  min_hash=float('inf')
  for i,bit in enumerate(characteristic_vec):
      if bit==1:
        index=h_ab(i+1)
        if index<min_hash:
          min_hash=index

  return min_hash


In [ ]:
# testing algo 4
# h_ab=F[random.randint(0,len(F)-1)]
characteristic_vec_eg=characteristic_vector_converting(taxonomy_dict, first_row, inverted_taxonomy_tree)
second_row = list(df.values[100])
second_row = [s.strip() for s in second_row if isinstance(s, str)]
vec1=characteristic_vector_converting(taxonomy_dict, first_row, inverted_taxonomy_tree)
vec2=characteristic_vector_converting(taxonomy_dict, second_row, inverted_taxonomy_tree)
min_hash1_val = minhash(vec1,F[2])
min_hash2_val= minhash(vec2,F[2])

print(min_hash1_val,min_hash2_val)

## TAXONOMY Helper funcs

In [ ]:
# find the least common ancestor
def find_lca(v1, v2, inverted_taxonomy_tree):
  if v1 == v2:
    return v1

# get paths for both values
  path1 = inverted_taxonomy_tree.get(v1, [])
  path2 = inverted_taxonomy_tree.get(v2, [])

  if not path1 or not path2:
    return None #different attributes have no common ancestor

  if path1[-1] != path2[-1]:
    return None

  # compare paths to find the lca"
  lca = None
  min_len = min(len(path1), len(path2))

  for i in range(1, min_len):
    idx = -i - 1
    if path1[idx] == path2[idx]:
      lca = path1[idx]
    else:
      break

  return lca

def get_tree_height(attr_name, inverted_taxonomy_tree):

  max_height = 0

  for value, path in inverted_taxonomy_tree.items():
        # Check if this value belongs to the attribute
      if path and path[-1] == attr_name:
            # Height = length of path minus 1 (exclude attribute name root)
          height = len(path) - 1
          max_height = max(max_height, height)

  return max_height

def path_length_between(v1, v2, inverted_taxonomy_tree):

    if v1 == v2:
        return 0

    lca = find_lca(v1, v2, inverted_taxonomy_tree)

    if lca is None:
        # No common ancestor meaning different attributes or invalid
        return float('inf')

    path1 = inverted_taxonomy_tree.get(v1, [])
    path2 = inverted_taxonomy_tree.get(v2, [])

    # Find distance from v1 to LCA
    # Count steps from v1 (index 0) to LCA
    try:
        lca_index_in_path1 = path1.index(lca)
        dist1 = lca_index_in_path1
    except ValueError:
        dist1 = 0

    # distance from v2 to LCA
    try:
        lca_index_in_path2 = path2.index(lca)
        dist2 = lca_index_in_path2
    except ValueError:
        dist2 = 0

    # Total path length
    L = dist1 + dist2

    return L




In [ ]:
# confirmations

v1 = "State-gov"
v2 = "Federal-gov"
lca = find_lca(v1, v2, inverted_taxonomy_tree)
print(f"LCA of '{v1}' and '{v2}': {lca}")
print(f"Expected: 'Government'")
print(f"Path1: {inverted_taxonomy_tree[v1]}")
print(f"Path2: {inverted_taxonomy_tree[v2]}")

v1 = "State-gov"
v2 = "Private"
lca = find_lca(v1, v2, inverted_taxonomy_tree)
print(f"\nLCA of '{v1}' and '{v2}': {lca}")
print(f"Expected: 'workclass' ")
print(f"Path1: {inverted_taxonomy_tree[v1]}")
print(f"Path2: {inverted_taxonomy_tree[v2]}")

In [ ]:
v1 = "Masters"
v2 = "Doctorate"
lca = find_lca(v1, v2, inverted_taxonomy_tree)
print(f"LCA of '{v1}' and '{v2}': {lca}")
print(f"Expected: 'Graduate'")
print(f"Path1: {inverted_taxonomy_tree[v1]}")
print(f"Path2: {inverted_taxonomy_tree[v2]}")

v1 = "Masters"
v2 = "Bachelors"
lca = find_lca(v1, v2, inverted_taxonomy_tree)
print(f"\nLCA of '{v1}' and '{v2}': {lca}")
print(f"Expected: 'University'")
print(f"Path1: {inverted_taxonomy_tree[v1]}")
print(f"Path2: {inverted_taxonomy_tree[v2]}")

v1 = "Masters"
v2 = "HS-grad"
lca = find_lca(v1, v2, inverted_taxonomy_tree)
print(f"\nLCA of '{v1}' and '{v2}': {lca}")
print(f"Expected: 'education' or 'Post-Secondary' and 'Secondary' common parent")
print(f"Path1: {inverted_taxonomy_tree[v1]}")
print(f"Path2: {inverted_taxonomy_tree[v2]}")

In [ ]:
# tree height

for attr in ['workclass', 'education', 'sex', 'race', 'marital-status', 'occupation']:
    height = get_tree_height(attr, inverted_taxonomy_tree)
    print(f"Height of '{attr}' taxonomy tree: {height}")

In [ ]:
# Same values
v1 = "Masters"
v2 = "Masters"
L = path_length_between(v1, v2, inverted_taxonomy_tree)
print(f"Path length between '{v1}' and '{v2}': {L}")
print(f"Expected: 0 (same value)")

In [ ]:
# too far apart
v1 = "Masters"
v2 = "HS-grad"
L = path_length_between(v1, v2, inverted_taxonomy_tree)
print(f"\nPath length between '{v1}' and '{v2}': {L}")
print(f"Path1: {inverted_taxonomy_tree[v1]}")
print(f"Path2: {inverted_taxonomy_tree[v2]}")

# Distance

In [ ]:
def categorical_distance(v1, v2, attr_name, inverted_taxonomy_tree):
    """
    Equation (5): Path-based distance between two categorical values
    d(v, v') = L(v, v') / (2H)

    """
    if v1 == v2:
        return 0.0

    # Get path length L(v1, v2)
    L = path_length_between(v1, v2, inverted_taxonomy_tree)

    if L == float('inf'):
        return 1.0

    # Get tree height H
    H = get_tree_height(attr_name, inverted_taxonomy_tree)

    if H == 0:
        return 0.0

    # Return normalized distance
    distance = L / (2.0 * H)

    return distance


def qid_distance(qid1, qid2, attr_names, inverted_taxonomy_tree, weights=None):
    """
    Equation (6): Distance between two quasi-identifiers (records)
    d(qid, qid') = Σ(ωᵢ × d(vᵢ, v'ᵢ))

    """
    m = len(qid1)

    if weights is None:
        weights = [1.0 / m] * m

    total_distance = 0.0

    for i in range(m):
        # Calculate distance for this attribute
        cat_dist = categorical_distance(
            qid1[i],
            qid2[i],
            attr_names[i],
            inverted_taxonomy_tree
        )
        total_distance += weights[i] * cat_dist

    return total_distance


def cluster_distance(cluster1, cluster2, k, attr_names, inverted_taxonomy_tree, theta=None):
    """
    Equation (8): Flexible distance between two clusters
    d(C, C') = (θ × Δ + 1) × max{d(qid, qid')}
    where Δ = |C| + |C'| - k

    """
    if theta is None:
        theta = 1.0 / k

    # Calculate Δ (delta)
    delta = len(cluster1) + len(cluster2) - k

    # Find maximum pairwise distance (diameter of merged cluster)
    max_distance = 0.0
    for qid1 in cluster1.records:
        for qid2 in cluster2.records:
            dist = qid_distance(qid1, qid2, attr_names, inverted_taxonomy_tree)
            max_distance = max(max_distance, dist)

    # Apply flexible distance formula
    flexible_distance = (theta * delta + 1) * max_distance

    return flexible_distance

In [ ]:
print("TEST 2: DISTANCE FUNCTIONS")

categorical_cols = ['workclass', 'education', 'marital-status', 'occupation',
                    'relationship', 'race', 'sex', 'native-country']

# Get first two records
record1 = df[categorical_cols].iloc[0].tolist()
record2 = df[categorical_cols].iloc[1].tolist()

# Strip whitespace
record1 = [s.strip() if isinstance(s, str) else s for s in record1]
record2 = [s.strip() if isinstance(s, str) else s for s in record2]

print("\n[Test 2.1] Categorical Distance")
print("-" * 50)
print(f"Record 1 workclass: {record1[0]}")
print(f"Record 2 workclass: {record2[0]}")

dist = categorical_distance(record1[0], record2[0], 'workclass', inverted_taxonomy_tree)
print(f"Categorical distance: {dist:.4f}")

print(f"\nRecord 1 education: {record1[1]}")
print(f"Record 2 education: {record2[1]}")
dist = categorical_distance(record1[1], record2[1], 'education', inverted_taxonomy_tree)
print(f"Categorical distance: {dist:.4f}")

print("\n[Test 2.2] QID Distance")
print("-" * 50)
print(f"Record 1: {record1}")
print(f"Record 2: {record2}")
dist = qid_distance(record1, record2, categorical_cols, inverted_taxonomy_tree)
print(f"QID distance: {dist:.4f}")

In [ ]:
class Cluster:
    """
    Represents a cluster of quasi-identifiers (data records)
    """

    def __init__(self, records):

        self.records = records

    def __len__(self):
        # Return number of records in cluster
        return len(self.records)

    def merge(self, other_cluster):

        # Merge two clusters into one

        return Cluster(self.records + other_cluster.records)

    def __repr__(self):
        return f"Cluster(size={len(self.records)})"

In [ ]:
print("TEST 3: CLUSTER CLASS")


# Create test clusters
records1 = [record1, record2]
records2 = df[categorical_cols].iloc[2:4].values.tolist()
records2 = [[s.strip() if isinstance(s, str) else s for s in rec] for rec in records2]

cluster1 = Cluster(records1)
cluster2 = Cluster(records2)

print(f"\nCluster 1: {cluster1}")
print(f"  Records: {len(cluster1.records)}")

print(f"\nCluster 2: {cluster2}")
print(f"  Records: {len(cluster2.records)}")

# Test merge
merged = cluster1.merge(cluster2)
print(f"\nMerged cluster: {merged}")
print(f"  Records: {len(merged.records)}")
print(f"Expected: 4 records")

In [ ]:
def _cluster_distance(self, cluster1, cluster2, k, theta):
        """
        Internal method to compute cluster distance using stored taxonomy tree
        """
        return cluster_distance(
            cluster1,
            cluster2,
            k,
            self.attribute_names,
            self.inverted_taxonomy_tree,
            theta
        )

def _qid_distance(self, qid1, qid2, weights=None):
        """
        Internal method to compute QID distance using stored taxonomy tree
        """
        return qid_distance(
            qid1,
            qid2,
            self.attribute_names,
            self.inverted_taxonomy_tree,
            weights
        )

def _categorical_distance(self, v1, v2, attribute_name):
        """
        Internal method to compute categorical distance using stored taxonomy tree
        """
        return categorical_distance(
            v1,
            v2,
            attribute_name,
            self.inverted_taxonomy_tree
        )

In [ ]:
import heapq

class BetaACClustering:

    def __init__(self, inverted_taxonomy_tree, attribute_names):
        """
        Args:
            inverted_taxonomy_tree: dict mapping values to taxonomy paths
                Example: {'Masters': ['Masters', 'Graduate', 'University', 'Post-Secondary', 'education']}
            attribute_names: list of attribute names in order
                Example: ['sex', 'zipcode', 'education', 'marital-status', ...]
        """
        self.inverted_taxonomy_tree = inverted_taxonomy_tree
        self.attribute_names = attribute_names

    def _cluster_distance(self, cluster1, cluster2, k, theta):
        """
        Internal method to compute cluster distance using stored taxonomy tree
        """
        return cluster_distance(
            cluster1,
            cluster2,
            k,
            self.attribute_names,
            self.inverted_taxonomy_tree,
            theta
        )

    # Strictly according to algorithm 6, line per line
    def beta_ac(self, small_clusters, k, theta=None):
        """
        Algorithm 6: Beta-AC (β-cluster Agglomerative Clustering)

        INPUT:
            small_clusters: list of Cluster objects (each size < k)
            k: privacy parameter (minimum cluster size)
            theta: weight parameter (default: 1/k)

        OUTPUT:
            tuple: (k_member_clusters, remaining_cluster)
                - k_member_clusters: list of Cluster objects (size >= k)
                - remaining_cluster: single Cluster object (size < k) or None
        """
        if theta is None:
            theta = 1.0 / k

        # Initialize outputs
        k_member_clusters = []
        remaining_cluster = None

        # Active clusters (use dict for easy deletion)
        active_clusters = {i: cluster for i, cluster in enumerate(small_clusters)}

        # Priority queue: (distance, cluster_id_1, cluster_id_2)
        pqueue = []

        # Line 1: Compute all pairwise distances and populate priority queue
        cluster_ids = list(active_clusters.keys())
        for i in range(len(cluster_ids)):
            for j in range(i + 1, len(cluster_ids)):
                id1, id2 = cluster_ids[i], cluster_ids[j]
                dist = self._cluster_distance(
                    active_clusters[id1],
                    active_clusters[id2],
                    k,
                    theta
                )
                heapq.heappush(pqueue, (dist, id1, id2))

        # Track next available cluster ID
        next_id = max(active_clusters.keys()) + 1 if active_clusters else 0

        # Line 2: Main merging loop
        while pqueue:
            # Line 3: Extract pair with minimum distance
            dist, id_x, id_y = heapq.heappop(pqueue)

            # Skip if either cluster was already merged
            if id_x not in active_clusters or id_y not in active_clusters:
                continue

            # Get clusters and merge
            cluster_x = active_clusters[id_x]
            cluster_y = active_clusters[id_y]
            cluster_z = cluster_x.merge(cluster_y)

            # Line 4: Remove old clusters from active set
            del active_clusters[id_x]
            del active_clusters[id_y]

            # Line 5: Delete entries from priority queue happens implicitly
            # (we skip invalid pairs in the loop above)

            # Line 6-10: Handle merged cluster based on size
            if len(cluster_z) >= k:
                # Line 7: Add to k-member clusters (done!)
                k_member_clusters.append(cluster_z)
            else:
                # Line 9: Add back to active clusters (needs more merging)
                new_id = next_id
                next_id += 1
                active_clusters[new_id] = cluster_z

                # Line 10: Update priority queue with new distances
                for other_id, other_cluster in active_clusters.items():
                    if other_id != new_id:
                        new_dist = self._cluster_distance(
                            cluster_z,
                            other_cluster,
                            k,
                            theta
                        )
                        heapq.heappush(pqueue, (new_dist, new_id, other_id))

        # Line 11: Handle remaining cluster (if exactly one left)
        if len(active_clusters) == 1:
            remaining_id = list(active_clusters.keys())[0]
            remaining_cluster = active_clusters[remaining_id]

        return k_member_clusters, remaining_cluster


In [ ]:
# Define attribute names (columns you're using for quasi-identifiers)
attribute_names = ['workclass', 'education', 'marital-status', 'occupation',
                   'relationship', 'race', 'sex', 'native-country']

# Initialize once
clusterer = BetaACClustering(inverted_taxonomy_tree, attribute_names)

# Prepare your data as Cluster objects
# Example: Convert dataframe rows to clusters
print(len(df))
small_clusters = []
for i in range(0, 1500, 2):  # Group every 2 records as a small cluster
    records = df.iloc[i:i+2][attribute_names].values.tolist()
    # Strip whitespace from strings
    records = [[s.strip() if isinstance(s, str) else s for s in record] for record in records]
    small_clusters.append(Cluster(records))

# Run Algorithm 6 (matches paper specification exactly!)
k = 10
k_member_clusters, remaining_cluster = clusterer.beta_ac(small_clusters, k)

# Check results
print(f"Created {len(k_member_clusters)} k-member clusters")
for i, cluster in enumerate(k_member_clusters):
    print(f"Cluster {i+1}: {len(cluster)} records")

In [ ]:
print("TEST 4: ALGORITHM 6 - BETA-AC")

# Prepare small clusters
k = 5
n_records = 20

sample_data = df[categorical_cols].iloc[:n_records].values.tolist()
sample_data = [[s.strip() if isinstance(s, str) else s for s in rec] for rec in sample_data]

# small clusters (each < k)
small_clusters = []
cluster_size = 2  # Each initial cluster has 2 records

for i in range(0, n_records, cluster_size):
    records = sample_data[i:i+cluster_size]
    if records:
        small_clusters.append(Cluster(records))

print(f"\n[Input]")
print(f"k = {k}")
print(f"Number of small clusters: {len(small_clusters)}")
print(f"Small cluster sizes: {[len(c) for c in small_clusters]}")

k_member_clusters, remaining_cluster = clusterer.beta_ac(
    small_clusters,
    k
)

print(f"\n[Output]")
print(f"Number of k-member clusters: {len(k_member_clusters)}")
print(f"k-member cluster sizes: {[len(c) for c in k_member_clusters]}")

if remaining_cluster:
    print(f"Remaining cluster size: {len(remaining_cluster)}")
else:
    print(f"No remaining cluster")

# Verify constraints
print(f"\n[Verification]")
all_valid = True
for i, cluster in enumerate(k_member_clusters):
    size = len(cluster)
    is_valid = k <= size <= 2*k - 1
    print(f"Cluster {i}: size={size}, valid={is_valid} (should be {k} <= size <= {2*k-1})")
    if not is_valid:
        all_valid = False

if remaining_cluster and len(remaining_cluster) >= k:
    print(f"WARNING: Remaining cluster has size >= k!")
    all_valid = False

print(f"\nAll constraints satisfied: {all_valid}")

# Check total records preserved
total_input = sum(len(c) for c in small_clusters)
total_output = sum(len(c) for c in k_member_clusters)
if remaining_cluster:
    total_output += len(remaining_cluster)

print(f"\nRecords in: {total_input}")
print(f"Records out: {total_output}")
print(f"All records preserved: {total_input == total_output}")

In [ ]:

# === Implementations of Algorithm 3 (Map) and Algorithm 5 (LSH-RC) using notebook functions ===
# This cell integrates with existing notebook variables/functions:
# - characteristic_vector_converting(taxonomy_dict, row_values, inverted_taxonomy_tree)
# - minhash(characteristic_vec, h_ab)  # h_ab is a single hash function from F
# - F : list of hash functions (length = alpha)
# - taxonomy_dict, inverted_taxonomy_tree : available taxonomy structures
# - alpha : banding parameter (number of hash rows used to form bucketID)
# - k : privacy parameter
#
# The functions below follow the paper's pseudocode and use the notebook's helpers.

from collections import defaultdict, deque
import heapq

def algorithm_3_map(record_id, row_values, taxonomy_dict, inverted_taxonomy_tree, F, alpha):
    """
    Implements Algorithm 3 (Map): convert record -> characteristic vector -> compute alpha MinHash values
    and concatenate them to create bucketID. Returns (bucketID, (record_id, row_values))
    """
    # convert to characteristic vector using existing function
    char_vec = characteristic_vector_converting(taxonomy_dict, row_values, inverted_taxonomy_tree)
    print(char_vec)
    # compute minhash for each hash function in F (use first alpha entries)
    if alpha <= 0:
        raise ValueError("alpha must be positive")
    if len(F) < alpha:
        raise ValueError("F must contain at least alpha hash functions")
    mins = []
    for i in range(alpha):
        h_ab = F[i]
        mh = minhash(char_vec, h_ab)
        mins.append(str(int(mh)))
    bucket_id = "-".join(mins)
    return bucket_id, (record_id, row_values, mins)

def lsh_partitioning_serial(records, taxonomy_dict, inverted_taxonomy_tree, F, alpha):
    """
    Serial emulation of Map+Shuffle: records is list of (record_id, row_values)
    Returns dict: bucket_id -> list of (record_id, row_values)
    """
    buckets = defaultdict(list)
    for rid, row in records:
        bid, payload = algorithm_3_map(rid, row, taxonomy_dict, inverted_taxonomy_tree, F, alpha)
        buckets[bid].append((rid, row))
    return buckets

# Helper: provenance set based jaccard distance pairwise using notebook provenance()
def provenance_set_of_row(row_values):
    # expects 'row_values' aligned with taxonomy order as used by characteristic_vector_converting
    # use existing 'provenance' helper that returns list of ancestors for each value
    ps = []
    for v in row_values:
        try:
            p = provenance([v])  # provenance returns list; we exclude original value in earlier helper, but for similarity consider combining root nodes
            # provenance() in this notebook returns mapping excluding original value; to be consistent with paper, include the value itself
            # so include v plus p
            set_elems = [v] + p
        except Exception:
            set_elems = [v]
        ps.extend(set_elems)
    return set(ps)

def jaccard_distance_sets(A, B):
    if not A and not B: return 0.0
    inter = len(A & B)
    uni = len(A | B)
    return 1.0 - inter/uni

def cluster_diameter(cluster):
    """
    cluster: list of (id, row_values)
    compute max pairwise provenance-set jaccard distance
    """
    provs = [provenance_set_of_row(row) for (_id,row) in cluster]
    maxd = 0.0
    for i in range(len(provs)):
        for j in range(i+1, len(provs)):
            d = jaccard_distance_sets(provs[i], provs[j])
            if d > maxd: maxd = d
    return maxd

# Beta-AC implementation following Algorithm 6
def beta_ac(small_clusters, k, theta=None):
    """
    small_clusters: list of clusters, each cluster is list of (id,row)
    returns (list_of_k_member_clusters, remaining_cluster_or_None)
    """
    if theta is None:
        theta = 1.0 / k
    # Initialize clusters
    clusters = [list(c) for c in small_clusters]
    n = len(clusters)
    if n == 0:
        return [], None
    # compute diameters
    diam = [cluster_diameter(c) for c in clusters]
    # priority queue of (distance, idx_a, idx_b)
    pq = []
    for i in range(n):
        for j in range(i+1, n):
            delta = len(clusters[i]) + len(clusters[j]) - k
            dist = (theta * max(delta,0) + 1) * max(diam[i], diam[j], 1e-12)
            heapq.heappush(pq, (dist, i, j))
    active = {i: clusters[i] for i in range(n)}
    removed = set()
    results = []
    next_idx = n
    # Maintain diam dict for new clusters as they are created
    diam_dict = {i: diam[i] for i in range(n)}
    while pq:
        dist, i, j = heapq.heappop(pq)
        if i in removed or j in removed:
            continue
        # merge i and j
        a = active.pop(i)
        b = active.pop(j)
        removed.add(i); removed.add(j)
        merged = a + b
        new_diam = cluster_diameter(merged)
        if len(merged) >= k:
            results.append(merged)
        else:
            # add back as active cluster with new index
            idx_new = next_idx; next_idx += 1
            active[idx_new] = merged
            diam_dict[idx_new] = new_diam
            # push distances against other active clusters
            for other_idx in list(active.keys()):
                if other_idx == idx_new: continue
                delta = len(active[other_idx]) + len(merged) - k
                dist2 = (theta * max(delta,0) + 1) * max(diam_dict[other_idx], new_diam, 1e-12)
                heapq.heappush(pq, (dist2, other_idx, idx_new))
    # choose remaining cluster if any active cluster < k remains
    remaining = None
    for c in active.values():
        if 0 < len(c) < k:
            remaining = c
            break
    return results, remaining

# Algorithm 5: LSH-RC recursive clustering
def algorithm_5_lsh_rc(C_records, k, alpha, taxonomy_dict, inverted_taxonomy_tree, F):
    """
    C_records: list of (id, row_values)
    Returns: (k_member_clusters_list, remaining_cluster_or_None)
    """
    C_out = []
    Cr = None
    CS = []  # small clusters to be processed by beta_ac
    if len(C_records) < k:
        return [], C_records
    if len(C_records) == k:
        return [C_records], None
    # partition using serial LSH partitioning
    buckets = lsh_partitioning_serial(C_records, taxonomy_dict, inverted_taxonomy_tree, F, alpha)
    for bid, bucket in buckets.items():
        if len(bucket) < k:
            CS.append(bucket)
        elif len(bucket) == k:
            C_out.append(bucket)
        else:
            # recursive call
            cprime, rem = algorithm_5_lsh_rc(bucket, k, alpha, taxonomy_dict, inverted_taxonomy_tree, F)
            if cprime:
                C_out.extend(cprime)
            if rem:
                CS.append(rem)
    # After processing buckets, run Beta-AC on CS
    if CS:
        ac_results, rem_cluster = beta_ac(CS, k)
        if ac_results:
            C_out.extend(ac_results)
        Cr = rem_cluster
    return C_out, Cr

# Expose names for easy usage
Algorithm3_map = algorithm_3_map
LSH_RC = algorithm_5_lsh_rc
Beta_AC = beta_ac

print("Algorithm 3 and Algorithm 5 implementations loaded: Algorithm3_map, LSH_RC, Beta_AC")


In [ ]:
import pprint as pp
records = [(str(i), list(df.iloc[i][categorical_cols])) for i in range(len(df))]

# Running algorithm 3
buckets = lsh_partitioning_serial(records, taxonomy_dict, inverted_taxonomy_tree, F, alpha)
len(buckets), [ (k,len(v)) for k,v in buckets.items()][:10]

# Running LSH-RC
kmembers, remaining = LSH_RC(records, k, alpha, taxonomy_dict, inverted_taxonomy_tree, F)
len(kmembers), remaining
